# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

### Install Prerequisites

To run the app in the notebook environment, you must first install the required packages by executing the two cells below. **Make sure to restart the kernel after running each cell.**

> Note: Restarting the kernel ensures that all installed dependencies are properly loaded into the environment.

In [2]:
# Please restart the notebook kernel after running this cell.
# ---- Instruction for SageMaker when session expires in the middle of something then switched to SageMaker
# import sys
# !{sys.executable} -m pip install --user -r requirements.txt # | grep -v "already satisfied"
#  ----- Instructions for Udacity workspace
!pip install --user -r requirements.txt # | grep -v "already satisfied"

     |████████████████████████████████| 37.1 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 19.3 MB 55.6 MB/s eta 0:00:01
     |████████████████████████████████| 21.0 MB 69.8 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 8.8 MB/s  eta 0:00:01
     |████████████████████████████████| 121 kB 85.5 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 80.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 70.3 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 83.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 73.1 MB/s eta 0:00:01
     |████████████████████████████████| 216 kB 72.4 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 12.9 MB/s eta 0:00:01


  Created wheel for bokeh: filename=bokeh-2.1.1-py3-none-any.whl size=9257186 sha256=0cc46783072593c0f45017edb06251e8b014cc4a1e17615d0ea1d0603a1c8f64
  Stored in directory: /root/.cache/pip/wheels/f7/55/ff/f3d7554e69382d31cf7ad857cf518af9b923134fca7d925187
Successfully built bokeh
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
# Please restart the notebook kernel after running this cell as well.
# ---- Instruction for SageMaker when session expires in the middle of something then switched to SageMaker
# import sys
# !{sys.executable} -m jupyter nbextension enable --py widgetsnbextension
#  ----- Instructions for Udacity workspace
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [16]:
# import ipywidgets
# ipywidgets.__version__
# '7.6.5'
# ---- Instruction for SageMaker when session expires in the middle of something then switched to SageMaker
import sys
!{sys.executable} -m pip install ipywidgets

### A Simple App

In this notebook we build a very simple app that uses our exported model.

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> Note how we are not importing anything from our source code (we do not use any module from the ``src`` directory). This is because the exported model, differently from the model weights, is a standalone serialization of our model and therefore it does not need anything else. You can ship that file to anybody, and as long as they can import ``torch``, they will be able to use your model. This is very important for releasing pytorch models to production.

### Test Your App
Go to a search engine for images (like Google Images) and search for images of some of the landmarks, like the Eiffel Tower, the Golden Gate Bridge, Machu Picchu and so on. Save a few examples locally, then upload them to your app to see how your model behaves!

The app will show the top 5 classes that the model think are most relevant for the picture you have uploaded

In [7]:
# !{sys.executable} -m jupyter labextension list
# !jupyter labextension list
# ---- Instruction for SageMaker when session expires in the middle of something then switched to SageMaker
import sys
!{sys.executable} -m jupyter labextension install @jupyter-widgets/jupyterlab-manager

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
Building jupyterlab assets (production, minimized)
The extension "@plotly/dash-jupyterlab" is outdated.

The extension "jupyterlab-dash" is outdated.



In [3]:
from ipywidgets import VBox, Button, FileUpload, Output, Label
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torchvision
import torchvision.transforms as T
import torch

# Decide which model you want to use among the ones exported
# learn_inf = torch.jit.load(# YOUR CODE HERE)
from src.predictor import Predictor
from src.helpers import compute_mean_and_std
from src.transfer import get_model_transfer_learning


# %matplotlib inline
from src.data import visualize_one_batch, get_data_loaders

# use get_data_loaders to get the data_loaders dictionary. Use a batch_size
# of 5, a validation size of 0.01 and num_workers=-1 (all CPUs)
data_loaders = get_data_loaders(batch_size=5, valid_size=0.01, num_workers=0) 


# Loading model using TorchScript
# learn_inf = torch.jit.load('generated-models/model_transfer.pt')
# Rebuild the model architecture
learn_inf = get_model_transfer_learning(model_name="resnet18", n_classes=50)

# Load the state_dict
learn_inf.load_state_dict(torch.load("generated-models/model_transfer.pt"))
learn_inf.eval()



# Attach class names and transforms to the loaded model
mean, std = compute_mean_and_std()  # ← add parentheses to call the function
learn_inf.class_names = data_loaders["train"].dataset.classes
learn_inf = Predictor(learn_inf, learn_inf.class_names, mean, std)


def on_click_classify(change):

    # Load image that has been uploaded
    fn = io.BytesIO(btn_upload.data[-1])

    img = Image.open(fn)
    img.load()

    # Let's clear the previous output (if any)
    out_pl.clear_output()

    # Display the image
    with out_pl:

        ratio = img.size[0] / img.size[1]
        c = img.copy()
        c.thumbnail([ratio * 200, 200])
        display(c)

    # Transform to tensor
    timg = T.ToTensor()(img).unsqueeze_(0)

    # Calling the model
    softmax = learn_inf(timg).data.cpu().numpy().squeeze()
    
    # Get the indexes of the classes ordered by softmax
    # (larger first)
    idxs = np.argsort(softmax)[::-1]
    
    # Loop over the classes with the largest softmax
    for i in range(5):
        # Get softmax value
        p = softmax[idxs[i]]
    
        # Get class name
        landmark_name = learn_inf.class_names[idxs[i]]
        
        labels[i].value = f"{landmark_name} (prob: {p:.2f})"


# Putting back btn_upload to a widget for next cell
btn_upload = FileUpload()

btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)

labels = []
for _ in range(5):
    labels.append(Label())

out_pl = Output()
out_pl.clear_output()

wgs = [Label("Please upload a picture of a landmark"), btn_upload, btn_run, out_pl]
wgs.extend(labels)

VBox(wgs)

## (Optional) Standalone App or Web App

You can run this notebook as a standalone app on your computer by following these steps:

1. Download this notebook in a directory on your machine
2. Download the model export (for example, ``checkpoints/transfer_exported.pt``) in a subdirectory called ``checkpoints`` within the directory where you save the app.ipynb notebook
3. Install voila if you don't have it already (``pip install voila``)
4. Run your app: ``voila app.ipynb --show_tracebacks=True``
5. Customize your notebook to make your app prettier and rerun voila

You can also deploy this app as a website using Binder: https://voila.readthedocs.io/en/stable/deploy.html#deployment-on-binder

# Create Your Submission Archive

Now that you are done with your project, please run the following cell. It will generate a file containing all the code you have written, as well as the notebooks. Please submit that file to complete your project

In [ ]:
!python src/create_submit_pkg.py